In [1]:
#ごめんなさい、これもだるすぎた
import json

file = []
with open('jawiki-country.json', 'r') as f:
    lines = f.readlines()
    for line in lines:
        file.append(json.loads(line))

for i in range(len(file)):
    if file[i]['title'] == "イギリス" :
        text_ = str(file[i])
        break

import re
pattern = '基礎情報(.*?\<references/\>)'
result = re.findall(pattern, text_)
result[0] += "\\n"
pattern = '(?<=\\\\n\|)(.*?) *= *(.*?)(?=\\\\n)'
result2 = re.findall(pattern, result[0])
inf_dic = {}
for i, j in result2:
  inf_dic[i] = j

inf_dic2 = {}
for key, text in inf_dic.items():
    inf_dic2[key] = re.sub(r'(\\\'){2,5}', '', text)

#　やる気でねぇわこんなの、
#　正規表現は覚えらんないからその都度調べようね

# 以下、他の人の回答
import re
inf_dic3 = {}
for key, text in inf_dic2.items():
  pattern = "(?<=\}\}\<br \/\>（)\[{2}"#<br />（フランス語]]:[[Dieu et mon droit|神と我が権利]]）
  text = re.sub(pattern, '', text)

  pattern = "\[{2}.*?\|.*?px\|(?=.*?\]\])"#'[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
  text = re.sub(pattern, '', text)

  pattern = "(?<=(\|))\[{2}"
  text = re.sub(pattern, '', text)

  pattern = "(?<=\}{2}（)\[{2}"#スコットランド・ゲール語]]）\n*{{lang|cy
  text = re.sub(pattern, '', text)

  pattern = "(?<=\>（)\[{2}.*?\|"#[[グレートブリテン及びアイルランド連合王国]]成立<br />（1800年合同法]]）
  text = re.sub(pattern, '', text)

  pattern = "(?<=（.{4}).*?\[{2}.*?\)\|" #'[[イングランド王国]]／[[スコットランド王国]]<br />（両国とも[[合同法 (1707年)|1707年合同法]]まで）',
  text = re.sub(pattern, '', text)

  pattern = "\[{2}.*?\|"#[[(除去)|]]の処理
  text = re.sub(pattern, '', text)

  pattern = "(\[{2}|\]{2})"#最後に残ったやつを処理
  inf_dic3[key] = re.sub(pattern, '', text)


inf_dic4 = {}
for key, text in inf_dic3.items():
  pattern = '\{\{.*?\{\{center\|' #{{center|ファイル:United States Navy Band - God Save the Queen.ogg}}',
  text = re.sub(pattern, '', text)

  pattern = '\{\{.*?\|.*?\|.{2}\|'
  text = re.sub(pattern, '', text) #'{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}'

  pattern = '\<ref.*?\>.*?\<\/ref\>'#<ref>で囲んでいるもの
  text = re.sub(pattern, '', text)

  pattern = '\<ref.*?\>|\<br \/\>'#<ref>単体+<br />単体
  text = re.sub(pattern, '', text)

  pattern = '\{\{lang\|.*?\|'#公式国名、標語の外国語タイトルを消すため
  text = re.sub(pattern, '', text)

  pattern = '\{\{.*?\}\}'#確立年月日2,3,4の{を除去する
  text = re.sub(pattern, '', text)

  pattern = '\}\}'# 公式国名、標語、国家、他元首等氏名2の}を除去する
  text = re.sub(pattern, '', text)

  inf_dic4[key] = text

import urllib
url = 'https://www.mediawiki.org/w/api.php?action=query&titles=File:' + urllib.parse.quote(inf_dic4['国旗画像']) + '&format=json&prop=imageinfo&iiprop=url'
connection = urllib.request.urlopen(urllib.request.Request(url))
response = json.loads(connection.read().decode())
print(response['query']['pages']['-1']['imageinfo'][0]['url'])


https://upload.wikimedia.org/wikipedia/commons/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg
